In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection 
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
# evaluate sklearn histogram gradient boosting algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
import random as rd
import timeit
import math
import sqlite3
from sqlalchemy import create_engine
import pickle
from xgboost import XGBClassifier
import shap
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score

In [3]:
os.chdir(r'C:\Users\formy\Downloads\amex-default-prediction')

In [3]:
sample_submission = pd.read_csv(r'sample_submission.csv')

In [4]:
# Sample Records
csv_database = sqlite3.connect('csv_database.db')
pd.read_sql_query('SELECT customer_ID FROM data_use limit 1 ',csv_database)
df = pd.read_sql_query('SELECT * FROM data_use where customer_ID = "00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7" ',csv_database)

In [23]:
df_ready.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
Index: 9990 entries, 0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba1730b2849b92ea687f9e to 059efd19407f9c2c43c58c65ed65be58ba7144d873846e8effd3dc55b2656dfa
Columns: 792 entries, P_2_mean to D_68_6.0_trend
dtypes: float64(792)
memory usage: 60.4+ MB


In [17]:
# loading customer id's
import pickle

with open("train", "rb") as fp:   #UnPickling train
     train_customers = pickle.load(fp)
        
with open("test", "rb") as fp:   #UnPickling test
     test_customers = pickle.load(fp)

In [32]:
def trainDataprep(step = 10000):
    j = 0
    while j<len(train_customers)-1:
        last_index = min(j+step,len(test_customers)-1)
        customer_slice = train_customers[j:last_index]
        j = last_index
        slice_query = 'SELECT * FROM raw_train_data WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
        df = pd.read_sql_query(slice_query,csv_database)
        df_dum = pd.get_dummies(df,columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #, weights='uniform', metric='nan_euclidean' 
        df_d = df_dum.dropna(axis='columns', thresh=0.4*len(df_dum.index))
        df_customers = df_d[['customer_ID']]
        df_d = df_d.drop(['customer_ID','S_2'], axis=1)
        imputer.fit(df_d)
        df_imp = pd.DataFrame(imputer.transform(df_d),columns = df_d.columns)
        df_imp['index1'] = df_imp.index
        df_customers['index1'] = df_customers.index
        df_final = df_imp.merge(df_customers)
        df_final = df_final.drop(['index','index1'], axis=1)
        #df_imp.index = df.index
        df_mean = df_final.groupby('customer_ID').agg(mean)
        df_var  = df_final.groupby('customer_ID').agg(stdev)
        df_range = df_final.groupby('customer_ID').agg(ranges)
        df_trend  = df_final.groupby('customer_ID').agg(trend)
        df_ready = df_mean.merge(df_var,on = ['customer_ID'],suffixes=('_mean', '_var')).merge(df_range,on = ['customer_ID'],suffixes=('', '_range')).merge(df_trend,on = ['customer_ID'],suffixes=('', '_trend'))
        df_ready.to_sql('train_data_refined',csv_database,if_exists = 'append')
        k = df_ready.index[-1]+1
        print('| index: {}'.format(k))
        
def testDataprep(step = 10000):
    j = 0
    while j<len(test_customers)-1:
        last_index = min(j+step,len(test_customers)-1)
        customer_slice = test_customers[j:last_index]
        j = last_index
        slice_query = 'SELECT * FROM data_use WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
        df = pd.read_sql_query(slice_query,csv_database)
        df_dum = pd.get_dummies(df,columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #, weights='uniform', metric='nan_euclidean' 
        df_d = df_dum.dropna(axis='columns', thresh=0.4*len(df_dum.index))
        df_customers = df_d[['customer_ID']]
        df_d = df_d.drop(['customer_ID','S_2'], axis=1)
        imputer.fit(df_d)
        df_imp = pd.DataFrame(imputer.transform(df_d),columns = df_d.columns)
        df_imp['index1'] = df_imp.index
        
        df_customers['index1'] = df_customers.index
        df_final = df_imp.merge(df_customers)
        df_final = df_final.drop(['index','index1'], axis=1)
        #df_imp.index = df.index
        df_mean = df_final.groupby('customer_ID').agg(mean)
        df_var  = df_final.groupby('customer_ID').agg(stdev)
        df_range = df_final.groupby('customer_ID').agg(ranges)
        df_trend  = df_final.groupby('customer_ID').agg(trend)
        df_ready = df_mean.merge(df_var,on = ['customer_ID'],suffixes=('_mean', '_var')).merge(df_range,on = ['customer_ID'],suffixes=('', '_range')).merge(df_trend,on = ['customer_ID'],suffixes=('', '_trend'))
        df_ready.to_sql('test_data_refined',csv_database,if_exists = 'append')
        k = df_ready.index[-1]+1
        print('| index: {}'.format(k))

In [12]:
df_train_final = pd.read_sql_query('select * from train_data_refined',csv_database)

In [13]:
df_labels = pd.read_csv('train_labels.csv')
df_train_with_labels = df_train_final.merge(df_labels)

In [10]:
df_train_with_labels

NameError: name 'df_train_with_labels' is not defined

In [71]:
# training the XGB model on the entire data 
df_train_with_labels = df_train_with_labels[variable_list_final]
X_train = df_train_with_labels.iloc[:,df_train_with_labels.columns != 'target']
y_train = df_train_with_labels[['target']]
model = XGBClassifier(objective = amex_metric)
#model.fit(X_train, y_train)

In [68]:
model = XGBClassifier()

In [72]:
scores = cross_val_score(model, X_train, y_train['target'].tolist(), cv=5,scoring = amex_metric)

C:\Users\formy\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\formy\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\formy\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\formy\anaconda3\lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\

In [57]:
scores

array([0.56530739, 0.56286261, 0.56372263, 0.56587831, 0.56580695])

In [18]:
customer_slice = test_customers[0:10]
slice_query = 'SELECT * FROM test_data_refined WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
test_data_slice = pd.read_sql_query(slice_query,csv_database)
variable_list = list(set(test_data_slice.columns) & set(df_train_final.columns))
variable_list.append('target')
variable_list_final = [i for i in variable_list if i != 'customer_ID']
variables = [i for i in variable_list_final if i != 'target']

In [19]:
# Saving the model
filename = 'XGBmodel'
XGB_model = pickle.load(open(filename, 'rb'))
df_train_with_labels = df_train_with_labels[variable_list_final]
X_train = df_train_with_labels.iloc[:,df_train_with_labels.columns != 'target']

In [36]:
Y_pred = model.predict(X_train)
Y_act = df_train_with_labels['target'].tolist()

In [37]:
sklearn.metrics.confusion_matrix(Y_act , Y_pred)

array([[322594,  17491],
       [ 18862,  99966]], dtype=int64)

In [66]:
def amex_metric(model,X, y_true):
    y_pred = model.predict(X)
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [39]:
amex_metric(Y_act, Y_pred)

0.6533488200028399

In [40]:
def Submissions(step = 100000):
    Submissions = pd.DataFrame(columns= ['customer_ID','prediction'])
    j = 0
    stepNum = 0
    while j<=len(test_customers):
        stepNum+=1
        last_index = min(j+step,len(test_customers))
        if last_index == j:
            break
        customer_slice = test_customers[j:last_index]
        j = last_index
        slice_query = 'SELECT * FROM test_data_refined WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
        test_data_slice = pd.read_sql_query(slice_query,csv_database)
        X_test = test_data_slice[variables]
        y_pred = model.predict(X_test)
        Submission_slice = pd.DataFrame()
        Submission_slice['customer_ID'] = test_data_slice['customer_ID']
        Submission_slice['prediction'] = y_pred
        Submissions = pd.concat([Submissions,Submission_slice])
        print(str(stepNum))
    Submissions.to_csv('Submission.csv')

In [41]:
Submissions()

1
2
3
4
5
6
7
8
9
10


In [104]:
Submissions = pd.DataFrame(columns= ['customer_ID','predictions'])
step = 1000
last_index = min(j+step,len(test_customers))
customer_slice = test_customers[j:last_index]
j = last_index
slice_query = 'SELECT * FROM test_data_refined WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
test_data_slice = pd.read_sql_query(slice_query,csv_database)
X_test = test_data_slice[variables]
y_pred = XGB_model.predict(X_test)
Submission_slice = pd.DataFrame()
Submission_slice['customer_ID'] = test_data_slice['customer_ID']
Submission_slice['predictions'] = y_pred
Submissions = pd.concat([Submissions,Submission_slice])

In [42]:
Submissions = pd.read_csv('Submission.csv')

In [44]:
Submissions

,Unnamed: 0,customer_ID,prediction
0,0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1
...,...,...,...
924615,24615,ffff824b313399b00db6bc930b83f1a2188d8b1dbd3a31...,0
924616,24616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,24617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,1
924618,24618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0


In [45]:
Submission = pd.DataFrame()
Submission['customer_ID'] = Submissions['customer_ID']
Submission['prediction'] = Submissions['prediction']
Submission.to_csv('Submission1.csv',index = False)

In [ ]:
sum(Submission['prediction'])/len(Submission['prediction'])